In [9]:
"""
                                Imports de bibliotecas necessáricas
"""

# Biblioteca para tratar vetores e matrizes
import numpy as np 

# Biblioteca para tratar a base de dados ou dataframe
import pandas as pd

# Bbilioteca que gera número aleatórios
import random



In [10]:
"""
                        Pega a base da dos e reduz ela a partir de atributos
"""
def get_atributos():

    # Carrega a base dados
    cl1 = pd.read_csv('https://raw.githubusercontent.com/SamuelHericles/knn_nearest_prototype_classifier/master/Classe1.csv')
    cl2 = pd.read_csv('https://raw.githubusercontent.com/SamuelHericles/knn_nearest_prototype_classifier/master/Classe2.csv')

    #Transposta da base dados
    cl1 = cl1.T
    cl2 = cl2.T

    # Rotular os dados
    cl1['labels'] = [1 for _ in range(0,cl1.shape[0])]
    cl2['labels'] = [2 for _ in range(0,cl2.shape[0])]

    # Armazena as duas classes em um unico dataframe
    base = cl1
    for i in range(0,cl2.shape[0]):
        base = base.append(cl2.iloc[i])
        base.reset_index(drop=True,inplace=True)

    # Reduz a base dados em atributos(caracteristicas definidas)
    atributos = pd.DataFrame({})
    atributos['media']    = base.iloc[:,:-1].mean(axis='columns')
    atributos['max']      = base.iloc[:,:-1].max(axis='columns')
    atributos['min']      = base.iloc[:,:-1].min(axis='columns')
    atributos['kurtosis'] = base.iloc[:,:-1].kurtosis(axis='columns')
    atributos['mediana']  = base.iloc[:,:-1].median(axis='columns')
    
    # Normlização zscore
    atributos = (atributos - atributos.mean())/atributos.var()

    # Rotulação dos dados
    atributos['labels']   = base['labels']

    # Retorna a base de dados   
    return atributos

In [11]:
# Calcula o ponto medio entre os pontos
def ponto_medio(x,y):
  return sum(x,y)/2

# Calcula a distancia eucliana entre os pontos
def dist_euclidiana(x,y):
  return np.sqrt(sum(pow(x-y,2)))

# Rotula os pontos e classifica os vizinhos mais proximos
def mode_labels(df,y_teste,k,j):
    label_mode = df.sort_values('distance')[:k]
    label_signal = int(label_mode['label'].mode().values[0])

    sample = pd.DataFrame(y_teste.iloc[j]).T
    sample['labels'] = label_signal

    return sample

# Calcula a acuracia dos resutlados
def acuracia(y_pred,y_teste):
  return  (sum(y_pred['labels'].values == y_teste['labels'].values)/y_pred.shape[0])*100


# Calcula os centroides da distribuicao de pontos de cada rotulo
def calcula_centroides(X):
  lb_1         = []
  lb_2         = []
  centroides   = pd.DataFrame(columns = X.columns[:-1])

  # Carregar cada ponto para o seu rótulo
  for i in range(0,X.shape[0]):
    if X['labels'].values[i] == 1:
      lb_1.append(X.iloc[i,:-1])
    else:
      lb_2.append(X.iloc[i,:-1])

  # Armazena os pontos dos centroides
  centroides = centroides.append(sum(lb_1)/len(lb_1),ignore_index=True)
  centroides = centroides.append(sum(lb_2)/len(lb_2),ignore_index=True)
  centroides.reset_index(drop=True,inplace=True)
  centroides['labels'] = [1,2]

  return centroides

In [12]:
"""
                    PRIMEIRO TRABALHO DE RECONHECIMENTO DE PADRÕES
                            UNIVERSIDADE FEDERAL DO CEARÁ
                            SAMUEL HERICLES SOUZA SILVEIRA
                                    SOBRAL - 2020

- Implementar os classificadores k-NN e Nearet Prototype Classifier
  para classificar o banco de dados fornecidos
- No algoritmo k-NN, você deverá testar vários valores de k e usar
  o valor que fornecer a melhor taxa de acerto
- Usar validação cruzada K-fold com K=10
- O classificador possui 2 classes: sinal de ECG e sinal de áudio
- Cada uma das 2 classes possui 50 sinais, cada sinal com duração de 500 pontos
- Você deve escolher os atributos que achar mais convenientes 
- Deve-se usar pelo menos 5 atributos.

"""


# Base de dados tratada
atributos = get_atributos()

"""
                                  Algoritmo K-Nearest-Neighborhood

1.Carregar os dados

2.Inicialize K no número de vizinhos escolhido

3. Para cada exemplo nos dados

3.1 Calcule a distância entre o exemplo da consulta e o exemplo atual dos dados.

3.2 Adicione a distância e o índice do exemplo a uma coleção ordenada

4. Classifique a coleção ordenada de distâncias e índices do menor para o maior
   (em ordem crescente) pelas distâncias

5. Escolha as primeiras K entradas da coleção classificada

6. Obtenha os rótulos das entradas K selecionadas

7. Retornar o moda dos rótulos K

"""


"""
  @param  X_treino    dados de treino com os rótulos(y_treino)
  @param  y_teste     dados de teste sem os rótulos
  @param  n_vizinhos  n vizinhos mais próximos escolhidos

  @return y_repd      retorna os rótulos dos y_teste para comparar com os reais
"""
def KNN(X_treino,y_teste,n_vizinhos):
  y_pred = pd.DataFrame({})

  # Calcular a distancia euclidiana para cada ponto do sinal de treino com o de cada teste
  for j in range(0,y_teste.shape[0]):
    df_aux = pd.DataFrame({'distance':[],'label':[]})  
    for i in range(0,X_treino.shape[0]):

      # Armazenar em um dataframe para tratamento posterior
      df_aux = df_aux.append({'distance':dist_euclidiana(X_treino.iloc[i][:-1],y_teste.iloc[j])
                      ,'label':X_treino['labels'].iloc[i]}
                      ,ignore_index=True)

    # Pegar os n vizinhos mais próximo e rotular o ponto de teste a partir da moda dos vizinhos                  
    y_pred = y_pred.append( mode_labels(df_aux,y_teste,n_vizinhos,j))

  return y_pred

"""
                                 Nearest Prototype Classifier

1. Carregar os dados

2. Dividir os dados de treino pelos rótulos

3. Calcular os pontos médios de cada rótulo

4. Calcular os pontos médios dos pontos médios de cada rótulo para obter o centroides

5. A partir dos centróides de cada rótulo calcular as distancias de cada ponto de teste

6. Cada ponto raquear os mais próximos

7. rótular os pontos de testes com os rótulos dos centroides mais próximos

"""

"""
  @param X dados de treino(possui os rótulos)
  @param y dados de teste(não possui os rótulos)

  @return rótulos dos pontos
"""
def Nearest_prototype_classifier(X,y):
  
  # Calcula os centróides dos rótulos
  centroides = calcula_centroides(X)

  # Calcula as distancias dos rótulos
  y_pred     = pd.DataFrame({'labels':[]})
  for i in range(0, y.shape[0]):
    df_aux = pd.DataFrame({'Distance':[],'labels':[]})
    for j in range(0,centroides.shape[0]):
      #print(f'{y.iloc[i,:-1]}-{centroides.iloc[j,:-1]}')
      df_aux = df_aux.append({'Distance': dist_euclidiana(y.iloc[i,:-1],
                                                             centroides.iloc[j,:-1]),
                              'labels'  : centroides.iloc[j,-1]},
                              ignore_index=True)    
    df_aux.sort_values('Distance',inplace=True)

    # Armazenza os rótulos dos pontos
    y_pred = y_pred.append({'labels':df_aux.iloc[0,1]}
                            ,ignore_index=True)
    y_pred.reset_index(drop=True,inplace=True)
    
  return y_pred

"""
                                        K-fold

Fornece índices de treinamento / teste para dividir dados em conjuntos de treinamento / 
teste. Divida o conjunto de dados em k dobras consecutivas        
"""

"""
  @param k             número de divisoes de trenio/teste
  @param atributos     base dados fornecida
  @param metodo        funçao de classificacao desejada
  @param n_vizinhos    numeros de vizinhos mair próximos, caso seja 0 não usado o knn

  @return               vazio, mas imprime na tela a acurácia média dos folds
"""
def kfold(k,atributos,metodo,n_vizinhos=0):

  # Carrega os indice da base da dados
  indices = [i for i in range(0,atributos.shape[0])]
  acuracias = []

  # Divide os dados para treino e teste a partir escolha aleatória
  for _ in range(0,int(len(indices)/k)):
    itreino = []
    iteste  = []

    # Escolhe os indices dos sinais de teste aleatoriamente
    values =  sorted([int(random.randint(0,99)) for _ in range(0,k)])

    # Os indices que nao sao pra teste vao para o treino
    for i in indices:
        if i in values:
          iteste.append(i)
        else:
          itreino.append(i)
    X = atributos.iloc[itreino,:]
    y = atributos.iloc[iteste,:]
    X.reset_index(drop=True,inplace=True)
    y.reset_index(drop=True,inplace=True)
    
    # Escolha do metodo de classificacao
    if n_vizinhos == 0:
      y_pred = metodo(X,y)
    else:
      y_pred = metodo(X,y.iloc[:,:-1],n_vizinhos)

    # Armazena cada acuracia dos folds  
    acuracias.append(acuracia(y,y_pred))
  
  # Exibe a acuracia media
  print(f'Acurácia média do {str(metodo.__name__)}: {np.mean(acuracias)}%')
  

"""
                      Teste para Nearest_prototype_classifier
"""
print('*'*80)
print(' '*20,'Teste com Nearest_prototype_classifier')
print('*'*80)
kfold(10,atributos,Nearest_prototype_classifier)
# Caso queria buscar o melhor kfold para o Nearest apenas descomente
# for i in [2,3,4,5,6,7,8,9,10,12,13,25]:
#   print(f'{i} folds temos:')
#   kfold(i,atributos,Nearest_prototype_classifier)
#   print('-'*50)

print('\n\n')

"""
                                 Teste para KNN
"""
print('*'*80)
print(' '*30,'Teste com KNN')
print('*'*80)
# Conforme requerido verificar o melhor resultado a partir dos parâmetros
for n_vizinhos in [2,3,4,5,6,10]:
    print(f'{n_vizinhos} vizinhos com 10 folds')
    kfold(10,atributos,KNN,n_vizinhos)
    print('-'*50)


********************************************************************************
                     Teste com Nearest_prototype_classifier
********************************************************************************
Acurácia média do Nearest_prototype_classifier: 83.55555555555556%



********************************************************************************
                               Teste com KNN
********************************************************************************
2 vizinhos com 10 folds
Acurácia média do KNN: 100.0%
--------------------------------------------------
3 vizinhos com 10 folds
Acurácia média do KNN: 98.0%
--------------------------------------------------
4 vizinhos com 10 folds
Acurácia média do KNN: 100.0%
--------------------------------------------------
5 vizinhos com 10 folds
Acurácia média do KNN: 97.88888888888889%
--------------------------------------------------
6 vizinhos com 10 folds
Acurácia média do KNN: 99.0%
-----------------